In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [20]:
IMG_SIZE = 190
BATCH_SIZE = 32

In [21]:
import splitfolders
data_dir = '../Brain_Project/brain_tumor_dataset'
splitfolders.ratio(data_dir,output='brain_dataset', ratio=(.8,.1,.1), seed=1337)

Copying files: 253 files [00:00, 858.86 files/s]


In [22]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=[0, 0.3]
)    

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [23]:
train_dir = '../Brain_Project/brain_dataset/train'
test_dir = '../Brain_Project/brain_dataset/test'
val_dir = '../Brain_Project/brain_dataset/val'

In [24]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

Found 202 images belonging to 2 classes.
Found 27 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [25]:
import tensorflow_hub as hub

Inception_V3 = hub.KerasLayer('https://tfhub.dev/google/imagenet/inception_v3/classification/5')

In [26]:
inputs = layers.Input(shape=(IMG_SIZE,IMG_SIZE,3))
x = Inception_V3(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(2, activation='softmax')(x)

model_0 = tf.keras.Model(inputs, outputs, name='model_0_V3_trainable')

In [27]:
model_0.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [28]:
model_0.summary()

Model: "model_0_V3_trainable"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 190, 190, 3)]     0         
_________________________________________________________________
keras_layer_1 (KerasLayer)   (None, 1001)              23853833  
_________________________________________________________________
dense_4 (Dense)              (None, 64)                64128     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 130       
Total params: 23,918,091
Trainable params: 64,258
Non-trainable params: 23,853,833
_________________________________________________________________


In [29]:
model_0_history = model_0.fit(
    train_generator,
    epochs=10,
    validation_data = val_generator
)

Epoch 1/10
7/7 [==============================] - 13s 792ms/step - loss: 1.3032 - accuracy: 0.6040 - val_loss: 2.2839 - val_accuracy: 0.3750
Epoch 2/10
7/7 [==============================] - 3s 385ms/step - loss: 0.9117 - accuracy: 0.6931 - val_loss: 1.8488 - val_accuracy: 0.4583
Epoch 3/10
7/7 [==============================] - 3s 421ms/step - loss: 0.7978 - accuracy: 0.6683 - val_loss: 1.8174 - val_accuracy: 0.4583
Epoch 4/10
7/7 [==============================] - 3s 337ms/step - loss: 0.6993 - accuracy: 0.7228 - val_loss: 1.7931 - val_accuracy: 0.4583
Epoch 5/10
7/7 [==============================] - 3s 394ms/step - loss: 0.5885 - accuracy: 0.7079 - val_loss: 1.7405 - val_accuracy: 0.5417
Epoch 6/10
7/7 [==============================] - 3s 378ms/step - loss: 0.6731 - accuracy: 0.7277 - val_loss: 1.6752 - val_accuracy: 0.5417
Epoch 7/10
7/7 [==============================] - 3s 375ms/step - loss: 0.5816 - accuracy: 0.7822 - val_loss: 1.7316 - val_accuracy: 0.5000
Epoch 8/10
7/7 [===

In [30]:
model_0.evaluate(test_generator)

1/1 [==============================] - 1s 1s/step - loss: 1.3130 - accuracy: 0.4444


[1.3129980564117432, 0.4444444477558136]

In [31]:
model_0_pred_probs = model_0.predict(test_generator)
model_0_pred_probs

array([[0.8693467 , 0.13065335],
       [0.04204755, 0.9579525 ],
       [0.9766758 , 0.02332421],
       [0.33742294, 0.66257703],
       [0.8431155 , 0.15688454],
       [0.0198065 , 0.98019356],
       [0.64538103, 0.35461897],
       [0.4117321 , 0.5882679 ],
       [0.2570822 , 0.7429178 ],
       [0.48984474, 0.51015526],
       [0.93620694, 0.0637931 ],
       [0.12843399, 0.87156606],
       [0.02198483, 0.9780151 ],
       [0.9151352 , 0.08486482],
       [0.5948723 , 0.40512767],
       [0.88573784, 0.11426212],
       [0.98240155, 0.01759843],
       [0.99464417, 0.00535581],
       [0.8408928 , 0.15910716],
       [0.98697704, 0.01302295],
       [0.5140867 , 0.48591325],
       [0.82793945, 0.1720605 ],
       [0.33290213, 0.6670979 ],
       [0.88377917, 0.11622088],
       [0.03440633, 0.9655937 ],
       [0.12483011, 0.8751699 ],
       [0.75566155, 0.24433844]], dtype=float32)

In [32]:
model_0_preds = tf.argmax(model_0_pred_probs, axis=1)
model_0_preds

<tf.Tensor: shape=(27,), dtype=int64, numpy=
array([0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0], dtype=int64)>

In [33]:
Inception_V3_untrainable = tf.keras.applications.inception_v3.InceptionV3(include_top=False)

for layer in Inception_V3_untrainable.layers:
    layer.trainable=True

In [34]:
inputs = layers.Input(shape=(IMG_SIZE,IMG_SIZE,3))
x = Inception_V3_untrainable(inputs)
x = layers.Dense(64, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(2, activation='softmax')(x)

model_1 = tf.keras.Model(inputs, outputs, name='model_1_V3_untrainable')

In [35]:
model_1.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [36]:
model_1.summary()

Model: "model_1_V3_untrainable"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 190, 190, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, None, None, 2048)  21802784  
_________________________________________________________________
dense_6 (Dense)              (None, 4, 4, 64)          131136    
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 130       
Total params: 21,934,050
Trainable params: 21,899,618
Non-trainable params: 34,432
_________________________________________________________________


In [37]:
model_1_history = model_1.fit(
    train_generator,
    epochs=10,
    validation_data = val_generator
)

Epoch 1/10
7/7 [==============================] - 19s 978ms/step - loss: 0.8694 - accuracy: 0.5495 - val_loss: 0.7483 - val_accuracy: 0.6250
Epoch 2/10
7/7 [==============================] - 3s 439ms/step - loss: 0.6732 - accuracy: 0.6089 - val_loss: 4.8025 - val_accuracy: 0.6250
Epoch 3/10
7/7 [==============================] - 3s 441ms/step - loss: 0.6721 - accuracy: 0.6089 - val_loss: 6.3389 - val_accuracy: 0.6250
Epoch 4/10
7/7 [==============================] - 3s 440ms/step - loss: 0.6104 - accuracy: 0.6287 - val_loss: 2.7954 - val_accuracy: 0.5417
Epoch 5/10
7/7 [==============================] - 3s 437ms/step - loss: 0.5788 - accuracy: 0.7079 - val_loss: 151.4320 - val_accuracy: 0.6250
Epoch 6/10
7/7 [==============================] - 3s 453ms/step - loss: 0.5637 - accuracy: 0.7228 - val_loss: 17.3001 - val_accuracy: 0.3750
Epoch 7/10
7/7 [==============================] - 3s 504ms/step - loss: 0.5146 - accuracy: 0.7574 - val_loss: 2260.8145 - val_accuracy: 0.3750
Epoch 8/10
7/